In [1]:
import pandas as pd
import numpy as np

In [2]:
# Loading all data bases 

branchements = pd.read_csv('../data_SUEZ/branchements.csv')
canalisations = pd.read_csv('../data_SUEZ/canalisations.csv')
evenements_train = pd.read_csv('../data_SUEZ/evenements_train.csv')
meteo_AlpesMaritimes = pd.read_csv('../data_SUEZ/meteo_AlpesMaritimes_pre_traite.csv')
nb_ev_train = pd.read_csv('../data_SUEZ/nb_ev_train.csv')
permut_train = pd.read_csv('../data_SUEZ/permut_train.csv')
stations_AlpesMaritimes = pd.read_csv('../data_SUEZ/stations_AlpesMaritimes.csv')

# Preprocessing

## Jointure

In [3]:
# Renaming columns

evenements_train.columns= ['GID_fuite', 'CODINSEE_fuite', 'date_fuite', 'GID_pose', 'COMPO_P', 'geometry_fuite']
branchements.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']
canalisations.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']

In [4]:
evenements_train['fuite'] = 1

In [5]:
# Différencier les GID des branchements et des canalisations

evenements_train['ajout_branch'] = '_b'
evenements_train['ajout_can'] = '_c'
evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'ajout_branch']
evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'ajout_can']
evenements_train.drop('ajout_branch', axis=1, inplace=True)
evenements_train.drop('ajout_can', axis=1, inplace=True)

branchements['ajout_branch'] = '_b'
branchements.GID_pose = branchements.GID_pose.map(str) + branchements.ajout_branch
branchements.drop('ajout_branch', axis=1, inplace=True)

canalisations['ajout_can'] = '_c'
canalisations.GID_pose = canalisations.GID_pose.map(str) + canalisations.ajout_can
canalisations.drop('ajout_can', axis=1, inplace=True)

In [6]:
# Creating COMPO_P column in canalisation and branchements 

elements = pd.concat([canalisations, branchements])

In [7]:
# Merge elements on evenements (creation BDD complète)

elements_train = pd.merge(elements, evenements_train, on = 'GID_pose', how = 'left')

## Feature engineering

In [8]:
# Utiliser le code INSEE de la fuite quand on l'a plutot que celui de la pose
elements_train.ix[~elements_train.CODINSEE_fuite.isnull(), 'CODINSEE_pose'] = elements_train.ix[~elements_train.CODINSEE_fuite.isnull(), 'CODINSEE_fuite']
elements_train.drop('CODINSEE_fuite', axis = 1, inplace = True)

In [10]:
def to_day(x):
    if (str(x) == 'NaT'):
        return np.nan
    else:
        return x.days

elements_train['age'] = pd.to_datetime('2016-12-31') - pd.to_datetime(elements_train['date_pose'])
elements_train.age = elements_train.age.map(to_day)

# Training

In [12]:
# Predict on submition dataset
fuite_columns = ['GID_fuite', 'date_fuite', 'geometry_fuite']
id_columns = ['GID_pose', 'CODINSEE_pose']
a_predir = ['fuite']
# mask = id_columns + fuite_columns + a_predir + ['geometry_pose']  # geometry_pose pas encore exploitable
mask = id_columns + fuite_columns + ['geometry_pose']  # geometry_pose pas encore exploitable

X_full = elements_train.drop(mask, axis = 1)
y_full = elements_train['fuite']


test_insee = [6003, 6004, 6016, 6030, 6070, 6095, 6108, 6112, 6116, 6138, 6152]
X = elements_train[elements_train['CODINSEE_pose'].isin(test_insee)]

# Submitting

In [88]:
# Generate submition dataset
submit_data = elements_train[elements_train['CODINSEE_pose'].isin(test_insee)]
submit_data = 

nan

In [91]:
# Generate submission


False